# Comprehensive Audiology Analysis Pipeline v4 - Claude Code Edition
**Research-Ready Analysis Tool for CRM, Vowel, and Consonant Perception Experiments**

## Overview
This notebook provides a complete, production-quality analysis pipeline for psychophysical hearing research data.

### Features:
- **Hard-coded configuration** for reproducible analysis
- **Advanced phonetic feature analysis** (Place, Manner, Voicing)
- **Comprehensive statistical testing** with confidence intervals and effect sizes
- **10 exploratory analyses** covering temporal trends, talker effects, correlation analyses
- **Publication-ready visualizations** with proper error bars and sample sizes
- **Granular CRM analysis** including gender-specific masking effects

### Instructions:
1. Edit the `CONFIGURATION` section below with your data path and CRM condition mappings
2. Run all cells sequentially
3. Review outputs and statistical tests

## 1. Setup and Configuration

In [ ]:
# ===== IMPORTS =====
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import ttest_ind, mannwhitneyu, shapiro, levene, pearsonr, spearmanr
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import os
import re
import glob
import warnings
from pathlib import Path
from collections import defaultdict
import itertools

warnings.filterwarnings('ignore')

# ===== VISUALIZATION SETTINGS =====
sns.set_theme(style="whitegrid", context="paper", font_scale=1.2)
plt.rcParams['figure.dpi'] = 150
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 10
plt.rcParams['axes.titlesize'] = 12
plt.rcParams['axes.labelsize'] = 11

print("✓ Libraries imported successfully")
print("✓ Visualization settings configured")

### CONFIGURATION SECTION
**CRITICAL: Edit this section before running the analysis**

In [ ]:
# ========================================
# USER CONFIGURATION
# ========================================

# 1. DATA DIRECTORY PATH
# Set this to the absolute path of your subject's data folder
DATA_DIR = "/path/to/your/data/folder"  # CHANGE THIS

# 2. CRM CONDITION MAPPING
# Map each CRM run number (extracted from filename like CI148_crm_3.txt) to its condition
# Available conditions: 'BM' (Bimodal), 'CI' (Cochlear Implant), 'HA' (Hearing Aid), 'Practice'
CRM_CONDITION_MAP = {
    0: 'Practice',
    1: 'BM',
    2: 'BM',
    3: 'BM',
    4: 'CI',
    5: 'HA',
    6: 'CI',
    7: 'HA',
    8: 'BM',
    9: 'HA',
    10: 'CI',
    # Add additional mappings as needed
}

# 3. PHONEME MAPPINGS (Standard - usually don't need to change)
VOWEL_MAP = {
    1: 'AE', 2: 'AH', 3: 'AW', 4: 'EH', 5: 'IH',
    6: 'IY', 7: 'OO', 8: 'UH', 9: 'UW'
}

CONSONANT_MAP = {
    1: '#', 2: '_', 3: 'b', 4: 'd', 5: 'f', 6: 'g',
    7: 'k', 8: 'm', 9: 'n', 10: '%', 11: 'p', 12: 's',
    13: 't', 14: 'v', 15: 'z', 16: '$'
}

# 4. PHONETIC FEATURE MAPPINGS (Miller-Nicely Framework)
# Format: phoneme: (voicing, place, manner)
# Voicing: 0=voiceless, 1=voiced
# Place: 0=labial, 1=alveolar, 2=velar, 3=palatal
# Manner: 0=stop, 1=nasal, 2=fricative, 3=affricate
CONSONANT_FEATURES = {
    'b': (1, 0, 0), 'p': (0, 0, 0),  # Labial stops
    'd': (1, 1, 0), 't': (0, 1, 0),  # Alveolar stops
    'g': (1, 2, 0), 'k': (0, 2, 0),  # Velar stops
    'm': (1, 0, 1), 'n': (1, 1, 1),  # Nasals
    'f': (0, 0, 2), 'v': (1, 0, 2),  # Labial fricatives
    's': (0, 1, 2), 'z': (1, 1, 2),  # Alveolar fricatives
    '#': (0, 3, 2), '_': (1, 3, 2),  # Palatal fricatives (sh, zh)
    '%': (0, 3, 3), '$': (1, 3, 3),  # Affricates (ch, j)
}

print("✓ Configuration loaded")
print(f"  Data directory: {DATA_DIR}")
print(f"  CRM conditions: {len(CRM_CONDITION_MAP)} runs mapped")
print(f"  Phoneme mappings: {len(VOWEL_MAP)} vowels, {len(CONSONANT_MAP)} consonants")

## 2. Helper Functions and Data Loading

In [ ]:
# ===== HELPER FUNCTIONS =====

def get_gender(talker_id):
    """Returns gender based on talker ID. IDs 0-3 are Male, 4-7 are Female."""
    if pd.isna(talker_id):
        return None
    return 'M' if int(talker_id) <= 3 else 'F'

def parse_crm_header(filepath):
    """Extracts talker and masker IDs from CRM file header."""
    try:
        with open(filepath, 'r') as f:
            header = f.readline()
        match = re.search(r'Talker (\d+), Maskers (\d+) and (\d+)', header)
        if match:
            return int(match.group(1)), int(match.group(2)), int(match.group(3))
    except Exception as e:
        print(f"Header parse error: {e}")
    return None, None, None

def calculate_srt_reversals(df_run):
    """
    Calculates Speech Reception Threshold using reversal method.
    Standard protocol: Average reversals 5-14 (skip first 4 for convergence).
    """
    snr = df_run['snr'].values
    correct = ((df_run['tc'] == df_run['rc']) & (df_run['tn'] == df_run['rn'])).values
    
    # Find reversals (correct/incorrect transitions)
    reversals = []
    prev = correct[0]
    for i in range(1, len(correct)):
        if correct[i] != prev:
            reversals.append(snr[i])
        prev = correct[i]
    
    if len(reversals) >= 5:
        # Use reversals 5-14 (indices 4-13)
        calc_revs = reversals[4:14] if len(reversals) >= 14 else reversals[4:]
        return np.mean(calc_revs), np.std(calc_revs), len(reversals)
    
    return np.nan, np.nan, len(reversals)

def calculate_ci_bootstrap(data, confidence=0.95, n_bootstrap=1000):
    """
    Calculates confidence interval using bootstrap resampling.
    Returns (mean, lower_bound, upper_bound, std_err).
    """
    data_clean = np.array([x for x in data if not np.isnan(x)])
    if len(data_clean) < 2:
        return np.nan, np.nan, np.nan, np.nan
    
    means = []
    for _ in range(n_bootstrap):
        sample = np.random.choice(data_clean, size=len(data_clean), replace=True)
        means.append(np.mean(sample))
    
    mean_val = np.mean(data_clean)
    alpha = 1 - confidence
    lower = np.percentile(means, (alpha/2) * 100)
    upper = np.percentile(means, (1 - alpha/2) * 100)
    stderr = np.std(data_clean) / np.sqrt(len(data_clean))
    
    return mean_val, lower, upper, stderr

def stat_test_auto(group1, group2, alpha=0.05):
    """
    Automatically selects and performs appropriate statistical test.
    Returns: (test_name, statistic, p_value, effect_size)
    """
    g1 = np.array([x for x in group1 if not np.isnan(x)])
    g2 = np.array([x for x in group2 if not np.isnan(x)])
    
    if len(g1) < 3 or len(g2) < 3:
        return "Insufficient data", np.nan, np.nan, np.nan
    
    # Test normality
    _, p1 = shapiro(g1) if len(g1) <= 5000 else (None, 1.0)  # Skip for large samples
    _, p2 = shapiro(g2) if len(g2) <= 5000 else (None, 1.0)
    
    # Cohen's d effect size
    pooled_std = np.sqrt(((len(g1)-1)*np.var(g1, ddof=1) + (len(g2)-1)*np.var(g2, ddof=1)) / (len(g1)+len(g2)-2))
    cohens_d = (np.mean(g1) - np.mean(g2)) / pooled_std if pooled_std > 0 else 0
    
    if p1 > alpha and p2 > alpha:
        # Parametric: Check equal variance
        _, p_var = levene(g1, g2)
        if p_var > alpha:
            stat, p = ttest_ind(g1, g2, equal_var=True)
            test_name = "Student's t-test"
        else:
            stat, p = ttest_ind(g1, g2, equal_var=False)
            test_name = "Welch's t-test"
    else:
        # Non-parametric
        stat, p = mannwhitneyu(g1, g2, alternative='two-sided')
        test_name = "Mann-Whitney U"
    
    return test_name, stat, p, cohens_d

print("✓ Helper functions defined")

### Data Loading

In [ ]:
# ===== LOAD DATA =====

def load_all_data(base_path, crm_map, vowel_map, cons_map):
    """
    Loads all experiment data from the specified directory.
    Returns: (df_crm, df_vowel, df_consonant, df_crm_summary)
    """
    base_path = Path(base_path)
    subject_id = base_path.name
    
    print(f"\n{'='*60}")
    print(f"Loading data for subject: {subject_id}")
    print(f"{'='*60}\n")
    
    # --- 1. LOAD CRM DATA ---
    crm_files = sorted(base_path.glob('*_crm_*.txt'))
    crm_data = []
    crm_summary = []
    
    print(f"Processing {len(crm_files)} CRM files...")
    
    for fpath in crm_files:
        # Extract run number
        match = re.search(r'_crm_(\d+)', fpath.name)
        if not match:
            continue
        run_id = int(match.group(1))
        condition = crm_map.get(run_id, 'Unknown')
        
        # Parse header for talker/masker info
        talker, m1, m2 = parse_crm_header(fpath)
        if talker is None:
            print(f"  Warning: Could not parse header for {fpath.name}")
            continue
        
        t_gender = get_gender(talker)
        m_gender = get_gender(m1)
        masker_type = "Same" if t_gender == m_gender else "Diff"
        gender_config = f"{t_gender}-{m_gender}{get_gender(m2)}"
        
        try:
            # Load trial data
            df_temp = pd.read_csv(fpath, sep='\s+', header=None, skiprows=2,
                                 names=['run', 'tc', 'rc', 'tn', 'rn', 'snr', 'rt'],
                                 on_bad_lines='skip')
            
            # Clean data (remove footer lines)
            df_temp = df_temp[pd.to_numeric(df_temp['run'], errors='coerce').notna()]
            df_temp = df_temp.astype({'tc': float, 'rc': float, 'tn': float, 'rn': float, 'snr': float, 'rt': float})
            
            # Add metadata
            df_temp['run_id'] = run_id
            df_temp['condition'] = condition
            df_temp['masker_type'] = masker_type
            df_temp['gender_config'] = gender_config
            df_temp['talker_gender'] = t_gender
            df_temp['masker_gender'] = m_gender
            df_temp['filename'] = fpath.name
            
            # Calculate correctness
            df_temp['correct'] = (df_temp['tc'] == df_temp['rc']) & (df_temp['tn'] == df_temp['rn'])
            
            # Classify error types
            conditions_err = [
                (df_temp['tc'] == df_temp['rc']) & (df_temp['tn'] == df_temp['rn']),
                (df_temp['tc'] == df_temp['rc']) & (df_temp['tn'] != df_temp['rn']),
                (df_temp['tc'] != df_temp['rc']) & (df_temp['tn'] == df_temp['rn'])
            ]
            choices_err = ['Correct', 'Number Error', 'Color Error']
            df_temp['error_type'] = np.select(conditions_err, choices_err, default='Both Error')
            
            # Add trial index within run
            df_temp['trial_in_run'] = range(1, len(df_temp) + 1)
            
            crm_data.append(df_temp)
            
            # Calculate SRT
            srt, sd, n_rev = calculate_srt_reversals(df_temp)
            
            crm_summary.append({
                'run_id': run_id,
                'filename': fpath.name,
                'condition': condition,
                'masker_type': masker_type,
                'gender_config': gender_config,
                'talker_gender': t_gender,
                'masker_gender': m_gender,
                'srt': srt,
                'srt_sd': sd,
                'n_reversals': n_rev,
                'n_trials': len(df_temp),
                'accuracy': df_temp['correct'].mean()
            })
            
        except Exception as e:
            print(f"  Error processing {fpath.name}: {e}")
            continue
    
    df_crm = pd.concat(crm_data, ignore_index=True) if crm_data else pd.DataFrame()
    df_crm_summary = pd.DataFrame(crm_summary) if crm_summary else pd.DataFrame()
    
    print(f"  ✓ CRM: {len(crm_summary)} runs loaded, {len(df_crm)} total trials")
    
    # --- 2. LOAD VOWEL DATA ---
    vowel_files = list(base_path.glob('*vow*.*txt')) + list(base_path.glob('*vow*.out'))
    vowel_data = []
    
    for vfile in vowel_files:
        condition = 'BM' if 'BM' in vfile.name else 'CI' if 'CI' in vfile.name else 'HA' if 'HA' in vfile.name else 'Unknown'
        try:
            df_v = pd.read_csv(vfile, sep='\s+', header=None,
                              names=['talker_id', 'target_id', 'response_id', 'score', 'rt'])
            df_v['condition'] = condition
            df_v['target_label'] = df_v['target_id'].map(vowel_map)
            df_v['response_label'] = df_v['response_id'].map(vowel_map)
            df_v['talker_gender'] = df_v['talker_id'].apply(get_gender)
            df_v['trial_idx'] = range(1, len(df_v) + 1)
            vowel_data.append(df_v)
        except Exception as e:
            print(f"  Warning: Could not load {vfile.name}: {e}")
    
    df_vowel = pd.concat(vowel_data, ignore_index=True) if vowel_data else pd.DataFrame()
    print(f"  ✓ Vowels: {len(vowel_files)} files loaded, {len(df_vowel)} total trials")
    
    # --- 3. LOAD CONSONANT DATA ---
    cons_files = list(base_path.glob('*cons*.*txt')) + list(base_path.glob('*cons*.out'))
    cons_data = []
    
    for cfile in cons_files:
        condition = 'BM' if 'BM' in cfile.name else 'CI' if 'CI' in cfile.name else 'HA' if 'HA' in cfile.name else 'Unknown'
        try:
            df_c = pd.read_csv(cfile, sep='\s+', header=None,
                              names=['talker_id', 'target_id', 'response_id', 'score', 'rt'])
            df_c['condition'] = condition
            df_c['target_label'] = df_c['target_id'].map(cons_map)
            df_c['response_label'] = df_c['response_id'].map(cons_map)
            df_c['talker_gender'] = df_c['talker_id'].apply(get_gender)
            df_c['trial_idx'] = range(1, len(df_c) + 1)
            cons_data.append(df_c)
        except Exception as e:
            print(f"  Warning: Could not load {cfile.name}: {e}")
    
    df_consonant = pd.concat(cons_data, ignore_index=True) if cons_data else pd.DataFrame()
    print(f"  ✓ Consonants: {len(cons_files)} files loaded, {len(df_consonant)} total trials")
    
    print(f"\n{'='*60}")
    print("Data loading complete!")
    print(f"{'='*60}\n")
    
    return df_crm, df_vowel, df_consonant, df_crm_summary

# Execute loading
df_crm, df_vowel, df_consonant, df_crm_summary = load_all_data(
    DATA_DIR, CRM_CONDITION_MAP, VOWEL_MAP, CONSONANT_MAP
)

## 3. Phonetic Feature Analysis (Miller-Nicely)

### Methodology:
We analyze consonant perception using the **Miller-Nicely (1955) framework**, which decomposes phonemes into three acoustic-phonetic features:

1. **Voicing**: Presence/absence of vocal fold vibration (low-frequency cue, <500 Hz)
2. **Place of Articulation**: Location in vocal tract where constriction occurs (high-frequency cue, >2000 Hz)
3. **Manner of Articulation**: Type of constriction (temporal/envelope cue)

**Clinical Relevance**: Feature transmission scores reveal which acoustic dimensions are accessible to the listener. Poor Place scores with intact Voicing suggest high-frequency hearing loss.

In [ ]:
# ===== EXPLORATORY ANALYSIS 1: PHONETIC FEATURE ANALYSIS =====

def analyze_phonetic_features(df, feature_map, title="Consonant"):
    """
    Performs rigorous phonetic feature analysis with confidence intervals.
    """
    if df.empty:
        print(f"No {title} data available for feature analysis.")
        return None
    
    # Filter to valid phonemes in feature map
    df_valid = df[
        df['target_label'].isin(feature_map.keys()) & 
        df['response_label'].isin(feature_map.keys())
    ].copy()
    
    if df_valid.empty:
        print(f"No valid {title} data for feature analysis.")
        return None
    
    # Extract features
    df_valid['target_voicing'] = df_valid['target_label'].apply(lambda x: feature_map[x][0])
    df_valid['target_place'] = df_valid['target_label'].apply(lambda x: feature_map[x][1])
    df_valid['target_manner'] = df_valid['target_label'].apply(lambda x: feature_map[x][2])
    df_valid['resp_voicing'] = df_valid['response_label'].apply(lambda x: feature_map[x][0])
    df_valid['resp_place'] = df_valid['response_label'].apply(lambda x: feature_map[x][1])
    df_valid['resp_manner'] = df_valid['response_label'].apply(lambda x: feature_map[x][2])
    
    # Calculate feature transmission
    results = []
    feature_names = ['Voicing', 'Place', 'Manner']
    
    print(f"\n{'='*60}")
    print(f"{title} Phonetic Feature Analysis")
    print(f"{'='*60}")
    print(f"Total valid trials: n={len(df_valid)}")
    
    for conditions in df_valid['condition'].unique():
        df_cond = df_valid[df_valid['condition'] == conditions]
        n = len(df_cond)
        
        print(f"\nCondition: {conditions} (n={n})")
        print("-" * 40)
        
        for feat_idx, feat_name in enumerate(feature_names):
            target_col = f'target_{feat_name.lower()}'
            resp_col = f'resp_{feat_name.lower()}'
            
            correct = (df_cond[target_col] == df_cond[resp_col]).astype(int)
            accuracy = correct.mean() * 100
            
            # Bootstrap CI
            mean_val, ci_lower, ci_upper, stderr = calculate_ci_bootstrap(correct * 100)
            
            results.append({
                'Condition': conditions,
                'Feature': feat_name,
                'Accuracy': accuracy,
                'CI_Lower': ci_lower,
                'CI_Upper': ci_upper,
                'Std_Err': stderr,
                'n': n
            })
            
            print(f"  {feat_name:12s}: {accuracy:5.1f}% [95% CI: {ci_lower:.1f}-{ci_upper:.1f}]")
    
    df_results = pd.DataFrame(results)
    
    # Visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    
    x_pos = np.arange(len(feature_names))
    width = 0.25
    conditions_list = df_results['Condition'].unique()
    
    for i, cond in enumerate(conditions_list):
        df_plot = df_results[df_results['Condition'] == cond]
        offset = (i - len(conditions_list)/2 + 0.5) * width
        
        yerr = np.array([df_plot['Accuracy'] - df_plot['CI_Lower'],
                        df_plot['CI_Upper'] - df_plot['Accuracy']])
        
        ax.bar(x_pos + offset, df_plot['Accuracy'], width,
               label=f"{cond} (n={df_plot['n'].iloc[0]})",
               yerr=yerr, capsize=5, alpha=0.8)
    
    ax.set_xlabel('Phonetic Feature')
    ax.set_ylabel('% Information Transmitted')
    ax.set_title(f'{title} Feature Transmission (Miller-Nicely Framework)\nError bars: 95% CI')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(feature_names)
    ax.set_ylim(0, 105)
    ax.legend()
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return df_results

# Execute analysis
feat_results_cons = analyze_phonetic_features(df_consonant, CONSONANT_FEATURES, "Consonant")

## 4. Confusion Matrix Analysis

### Methodology:
Confusion matrices reveal **systematic perceptual errors**. The diagonal represents correct identifications, while off-diagonal cells show specific confusions.

We present two views:
- **Raw Counts**: Absolute number of responses
- **Row-Normalized Probabilities**: P(Response|Target)

In [ ]:
# ===== EXPLORATORY ANALYSIS 2: CONFUSION MATRICES WITH PROPER TYPE HANDLING =====

def plot_confusion_matrix_dual(df, title="Phoneme", condition_filter=None):
    """
    Creates dual confusion matrix (counts and probabilities) with proper NaN handling.
    """
    if df.empty:
        print(f"No {title} data available.")
        return
    
    # Filter by condition if specified
    if condition_filter:
        df = df[df['condition'] == condition_filter].copy()
        title = f"{title} ({condition_filter})"
    
    # CRITICAL: Remove NaN values and ensure strings
    df_clean = df.dropna(subset=['target_label', 'response_label']).copy()
    df_clean['target_label'] = df_clean['target_label'].astype(str)
    df_clean['response_label'] = df_clean['response_label'].astype(str)
    
    if df_clean.empty:
        print(f"No valid {title} data after cleaning.")
        return
    
    # Get sorted unique labels
    all_labels = sorted(list(set(df_clean['target_label'].unique()) | set(df_clean['response_label'].unique())))
    
    # Create confusion matrices
    cm_counts = pd.crosstab(df_clean['target_label'], df_clean['response_label'])
    cm_counts = cm_counts.reindex(index=all_labels, columns=all_labels, fill_value=0)
    
    # Row-normalized probabilities
    cm_prob = cm_counts.div(cm_counts.sum(axis=1), axis=0).fillna(0)
    
    # Calculate overall accuracy
    n_total = cm_counts.sum().sum()
    n_correct = np.trace(cm_counts.values)
    accuracy = (n_correct / n_total * 100) if n_total > 0 else 0
    
    print(f"\n{title} Confusion Matrix: n={n_total}, Accuracy={accuracy:.1f}%")
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(20, 8))
    
    # Counts
    sns.heatmap(cm_counts, annot=True, fmt='d', cmap='Blues', ax=axes[0],
                cbar=False, square=True, linewidths=0.5)
    axes[0].set_title(f'{title} - Raw Counts (n={n_total})')
    axes[0].set_ylabel('Target', fontweight='bold')
    axes[0].set_xlabel('Response', fontweight='bold')
    
    # Probabilities
    sns.heatmap(cm_prob, annot=True, fmt='.2f', cmap='Reds', ax=axes[1],
                vmin=0, vmax=1, square=True, linewidths=0.5, cbar_kws={'label': 'P(Response|Target)'})
    axes[1].set_title(f'{title} - Conditional Probabilities (Accuracy={accuracy:.1f}%)')
    axes[1].set_ylabel('')
    axes[1].set_xlabel('Response', fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Execute for vowels and consonants
print("\n" + "="*60)
print("VOWEL CONFUSION ANALYSIS")
print("="*60)
for cond in df_vowel['condition'].unique():
    plot_confusion_matrix_dual(df_vowel, "Vowel", condition_filter=cond)

print("\n" + "="*60)
print("CONSONANT CONFUSION ANALYSIS")
print("="*60)
plot_confusion_matrix_dual(df_consonant, "Consonant")

## 5. CRM Analysis: Speech Reception Thresholds (SRT)

### Methodology:
**SRT (Speech Reception Threshold)**: The signal-to-noise ratio (dB SNR) at which the subject achieves 50% accuracy, calculated via adaptive tracking with reversals.

**Lower SRT = Better performance** (less signal needed relative to noise)

In [ ]:
# ===== EXPLORATORY ANALYSIS 4: GRANULAR CRM ANALYSIS =====

print("\n" + "="*60)
print("CRM SPEECH RECEPTION THRESHOLD (SRT) ANALYSIS")
print("="*60)

if df_crm_summary.empty:
    print("No CRM data available.")
else:
    # Filter valid data (exclude Practice and Unknown)
    df_valid = df_crm_summary[
        (~df_crm_summary['condition'].isin(['Practice', 'Unknown'])) &
        (df_crm_summary['srt'].notna())
    ].copy()
    
    print(f"\nValid CRM runs: n={len(df_valid)}")
    print(f"Conditions: {', '.join(df_valid['condition'].unique())}")
    print(f"\nSummary Statistics by Condition:")
    print("-" * 60)
    
    # Calculate statistics with CI
    for cond in sorted(df_valid['condition'].unique()):
        df_cond = df_valid[df_valid['condition'] == cond]
        n = len(df_cond)
        mean_val, ci_lower, ci_upper, stderr = calculate_ci_bootstrap(df_cond['srt'].values)
        
        print(f"\n{cond}:")
        print(f"  n = {n}")
        print(f"  SRT = {mean_val:.2f} dB [95% CI: {ci_lower:.2f} to {ci_upper:.2f}]")
        print(f"  SD = {df_cond['srt'].std():.2f} dB")
    
    # --- PLOT 1: Global Distribution ---
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Violin + strip
    parts = ax.violinplot([df_valid[df_valid['condition'] == c]['srt'].dropna().values 
                           for c in sorted(df_valid['condition'].unique())],
                          positions=range(len(df_valid['condition'].unique())),
                          widths=0.7, showmeans=False, showextrema=False)
    
    for pc in parts['bodies']:
        pc.set_facecolor('lightgray')
        pc.set_alpha(0.5)
    
    # Overlay points with error bars
    for i, cond in enumerate(sorted(df_valid['condition'].unique())):
        df_cond = df_valid[df_valid['condition'] == cond]
        x = np.random.normal(i, 0.04, size=len(df_cond))
        ax.scatter(x, df_cond['srt'], alpha=0.6, s=100, edgecolors='black', linewidths=1.5)
        
        # Add mean with CI
        mean_val, ci_lower, ci_upper, _ = calculate_ci_bootstrap(df_cond['srt'].values)
        ax.errorbar(i, mean_val, yerr=[[mean_val-ci_lower], [ci_upper-mean_val]], 
                   fmt='D', color='red', markersize=10, linewidth=2, capsize=10, capthick=2)
    
    ax.set_xticks(range(len(df_valid['condition'].unique())))
    ax.set_xticklabels(sorted(df_valid['condition'].unique()))
    ax.set_ylabel('SRT (dB SNR) - Lower is Better', fontweight='bold')
    ax.set_xlabel('Condition', fontweight='bold')
    ax.set_title('Speech Reception Thresholds by Condition\nRed diamonds: Mean ± 95% CI', fontsize=14)
    ax.axhline(0, color='black', linestyle='--', alpha=0.3, linewidth=1)
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # --- PLOT 2: Condition × Masker Type ---
    fig, ax = plt.subplots(figsize=(12, 6))
    
    conditions = sorted(df_valid['condition'].unique())
    masker_types = sorted(df_valid['masker_type'].unique())
    x = np.arange(len(conditions))
    width = 0.35
    
    for i, mtype in enumerate(masker_types):
        means = []
        errs_lower = []
        errs_upper = []
        ns = []
        
        for cond in conditions:
            df_subset = df_valid[(df_valid['condition'] == cond) & (df_valid['masker_type'] == mtype)]
            if len(df_subset) > 0:
                mean_val, ci_lower, ci_upper, _ = calculate_ci_bootstrap(df_subset['srt'].values)
                means.append(mean_val)
                errs_lower.append(mean_val - ci_lower)
                errs_upper.append(ci_upper - mean_val)
                ns.append(len(df_subset))
            else:
                means.append(0)
                errs_lower.append(0)
                errs_upper.append(0)
                ns.append(0)
        
        offset = (i - 0.5) * width
        bars = ax.bar(x + offset, means, width, label=f"{mtype} Gender",
                     yerr=[errs_lower, errs_upper], capsize=5, alpha=0.8)
        
        # Add sample sizes on bars
        for j, (bar, n) in enumerate(zip(bars, ns)):
            if n > 0:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height + errs_upper[j] + 0.5,
                       f'n={n}', ha='center', va='bottom', fontsize=9)
    
    ax.set_ylabel('SRT (dB SNR)', fontweight='bold')
    ax.set_xlabel('Listening Condition', fontweight='bold')
    ax.set_title('SRT by Condition and Masker Gender Configuration\nError bars: 95% CI', fontsize=14)
    ax.set_xticks(x)
    ax.set_xticklabels(conditions)
    ax.legend()
    ax.axhline(0, color='black', linestyle='--', alpha=0.3, linewidth=1)
    ax.grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 6. Voice Gender Release from Masking (VGRM)

### Methodology:
**VGRM** quantifies the benefit of having a target talker with a different gender than the maskers.

$$VGRM = SRT_{Same Gender} - SRT_{Different Gender}$$

**Positive VGRM = Benefit** (lower SRT needed when genders differ)

In [ ]:
# ===== VOICE GENDER RELEASE FROM MASKING (VGRM) =====

print("\n" + "="*60)
print("VOICE GENDER RELEASE FROM MASKING (VGRM) ANALYSIS")
print("="*60)

if not df_valid.empty:
    vgrm_results = []
    
    print("\nVGRM Calculation (Same - Different):")
    print("-" * 60)
    
    for cond in sorted(df_valid['condition'].unique()):
        same_srt = df_valid[(df_valid['condition'] == cond) & (df_valid['masker_type'] == 'Same')]['srt']
        diff_srt = df_valid[(df_valid['condition'] == cond) & (df_valid['masker_type'] == 'Diff')]['srt']
        
        if len(same_srt) > 0 and len(diff_srt) > 0:
            vgrm = same_srt.mean() - diff_srt.mean()
            
            # Statistical test
            test_name, stat, p, cohens_d = stat_test_auto(same_srt.values, diff_srt.values)
            
            vgrm_results.append({
                'Condition': cond,
                'VGRM_dB': vgrm,
                'Same_mean': same_srt.mean(),
                'Diff_mean': diff_srt.mean(),
                'n_same': len(same_srt),
                'n_diff': len(diff_srt),
                'p_value': p,
                'significant': p < 0.05,
                'effect_size': cohens_d
            })
            
            sig_marker = "***" if p < 0.001 else "**" if p < 0.01 else "*" if p < 0.05 else "n.s."
            
            print(f"\n{cond}:")
            print(f"  Same Gender SRT:  {same_srt.mean():6.2f} dB (n={len(same_srt)})")
            print(f"  Diff Gender SRT:  {diff_srt.mean():6.2f} dB (n={len(diff_srt)})")
            print(f"  VGRM Benefit:     {vgrm:6.2f} dB {sig_marker}")
            print(f"  Statistical Test: {test_name}, p={p:.4f}, d={cohens_d:.2f}")
    
    # Plot VGRM
    if vgrm_results:
        df_vgrm = pd.DataFrame(vgrm_results)
        
        fig, ax = plt.subplots(figsize=(10, 6))
        
        colors = ['green' if x > 0 else 'red' for x in df_vgrm['VGRM_dB']]
        bars = ax.bar(range(len(df_vgrm)), df_vgrm['VGRM_dB'], color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
        
        # Add significance stars
        for i, row in df_vgrm.iterrows():
            marker = "***" if row['p_value'] < 0.001 else "**" if row['p_value'] < 0.01 else "*" if row['p_value'] < 0.05 else "n.s."
            y_pos = row['VGRM_dB'] + (0.5 if row['VGRM_dB'] > 0 else -0.5)
            ax.text(i, y_pos, marker, ha='center', va='bottom' if row['VGRM_dB'] > 0 else 'top', fontsize=14, fontweight='bold')
            
            # Add effect size
            ax.text(i, 0.1, f"d={row['effect_size']:.2f}", ha='center', va='bottom', fontsize=9)
        
        ax.set_xticks(range(len(df_vgrm)))
        ax.set_xticklabels(df_vgrm['Condition'])
        ax.set_ylabel('VGRM Benefit (dB)', fontweight='bold')
        ax.set_xlabel('Listening Condition', fontweight='bold')
        ax.set_title('Voice Gender Release from Masking\nPositive = Benefit from Gender Difference\n*p<0.05, **p<0.01, ***p<0.001', fontsize=14)
        ax.axhline(0, color='black', linestyle='-', linewidth=2)
        ax.grid(axis='y', alpha=0.3)
        
        plt.tight_layout()
        plt.show()

## 7. Talker-Specific Performance Analysis

### Exploratory Analysis 3:
Examines whether performance varies by talker characteristics (individual talker, gender, pitch)

In [ ]:
# ===== EXPLORATORY ANALYSIS 3: TALKER-SPECIFIC PERFORMANCE =====

print("\n" + "="*60)
print("TALKER-SPECIFIC PERFORMANCE ANALYSIS")
print("="*60)

# --- VOWEL ANALYSIS BY TALKER ---
if not df_vowel.empty:
    print("\nVowel Accuracy by Individual Talker:")
    print("-" * 60)
    
    talker_stats = []
    for talker in sorted(df_vowel['talker_id'].unique()):
        df_talk = df_vowel[df_vowel['talker_id'] == talker]
        acc = df_talk['score'].mean() * 100
        n = len(df_talk)
        gender = df_talk['talker_gender'].iloc[0]
        
        mean_val, ci_lower, ci_upper, _ = calculate_ci_bootstrap(df_talk['score'].values * 100)
        
        talker_stats.append({
            'Talker': int(talker),
            'Gender': gender,
            'Accuracy': acc,
            'CI_Lower': ci_lower,
            'CI_Upper': ci_upper,
            'n': n
        })
        
        print(f"Talker {int(talker):2d} ({gender}): {acc:5.1f}% [95% CI: {ci_lower:.1f}-{ci_upper:.1f}], n={n}")
    
    df_talker = pd.DataFrame(talker_stats)
    
    # Plot by talker
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Individual talkers
    colors = ['steelblue' if g == 'M' else 'coral' for g in df_talker['Gender']]
    axes[0].bar(range(len(df_talker)), df_talker['Accuracy'], 
               yerr=[df_talker['Accuracy'] - df_talker['CI_Lower'],
                     df_talker['CI_Upper'] - df_talker['Accuracy']],
               capsize=3, color=colors, alpha=0.7, edgecolor='black')
    axes[0].set_xticks(range(len(df_talker)))
    axes[0].set_xticklabels([f"T{t}" for t in df_talker['Talker']], rotation=45)
    axes[0].set_ylabel('Accuracy (%)', fontweight='bold')
    axes[0].set_xlabel('Talker ID', fontweight='bold')
    axes[0].set_title('Vowel Accuracy by Individual Talker\nBlue=Male, Orange=Female')
    axes[0].set_ylim(0, 105)
    axes[0].grid(axis='y', alpha=0.3)
    
    # By gender aggregate
    for gender in ['M', 'F']:
        df_gender = df_vowel[df_vowel['talker_gender'] == gender]
        acc_vals = df_gender['score'].values * 100
        mean_val, ci_lower, ci_upper, _ = calculate_ci_bootstrap(acc_vals)
        
        idx = 0 if gender == 'M' else 1
        color = 'steelblue' if gender == 'M' else 'coral'
        axes[1].bar(idx, mean_val, yerr=[[mean_val-ci_lower], [ci_upper-mean_val]],
                   capsize=10, color=color, alpha=0.7, edgecolor='black', linewidth=2)
        axes[1].text(idx, mean_val + (ci_upper-mean_val) + 2, f"n={len(df_gender)}",
                    ha='center', va='bottom', fontsize=10, fontweight='bold')
    
    axes[1].set_xticks([0, 1])
    axes[1].set_xticklabels(['Male Talkers', 'Female Talkers'])
    axes[1].set_ylabel('Accuracy (%)', fontweight='bold')
    axes[1].set_title('Vowel Accuracy by Talker Gender\nError bars: 95% CI')
    axes[1].set_ylim(0, 105)
    axes[1].grid(axis='y', alpha=0.3)
    
    # Statistical test
    male_acc = df_vowel[df_vowel['talker_gender'] == 'M']['score'].values
    female_acc = df_vowel[df_vowel['talker_gender'] == 'F']['score'].values
    test_name, stat, p, cohens_d = stat_test_auto(male_acc, female_acc)
    
    sig_text = "*" if p < 0.05 else "n.s."
    axes[1].text(0.5, max(mean_val + 5, 95), f"{test_name}\np={p:.4f} {sig_text}",
                ha='center', va='top', fontsize=9, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()
    
    print(f"\nGender Comparison: {test_name}, p={p:.4f}, Cohen's d={cohens_d:.2f}")

## 8. Comprehensive Statistical Testing

### Exploratory Analysis 5:
Performs ANOVA and post-hoc tests to determine statistical significance of observed differences

In [ ]:
# ===== EXPLORATORY ANALYSIS 5: ANOVA AND POST-HOC TESTS =====

print("\n" + "="*60)
print("COMPREHENSIVE STATISTICAL TESTING")
print("="*60)

if not df_valid.empty and len(df_valid['condition'].unique()) > 2:
    print("\n1. ONE-WAY ANOVA: SRT ~ Condition")
    print("-" * 60)
    
    # Prepare data for ANOVA
    groups = [df_valid[df_valid['condition'] == c]['srt'].dropna().values 
             for c in df_valid['condition'].unique()]
    
    # Perform ANOVA
    f_stat, p_anova = stats.f_oneway(*groups)
    
    print(f"F-statistic: {f_stat:.3f}")
    print(f"p-value: {p_anova:.4f}")
    
    if p_anova < 0.05:
        print("Result: SIGNIFICANT main effect of Condition (p < 0.05)")
        print("\nProceeding with POST-HOC pairwise comparisons (Tukey HSD)...")
        
        # Tukey HSD
        df_anova = df_valid[['srt', 'condition']].dropna()
        tukey = pairwise_tukeyhsd(endog=df_anova['srt'], groups=df_anova['condition'], alpha=0.05)
        
        print(tukey)
        print("\nNote: 'reject=True' indicates significant difference between conditions")
    else:
        print("Result: No significant main effect of Condition (p >= 0.05)")
    
    # Two-way ANOVA if we have both condition and masker type
    if len(df_valid['masker_type'].unique()) > 1:
        print("\n2. TWO-WAY ANOVA: SRT ~ Condition + MaskerType + Interaction")
        print("-" * 60)
        
        try:
            model = ols('srt ~ C(condition) * C(masker_type)', data=df_valid).fit()
            anova_table = sm.stats.anova_lm(model, typ=2)
            print(anova_table)
            
            print("\nInterpretation:")
            for idx, row in anova_table.iterrows():
                if 'Residual' not in str(idx):
                    p_val = row['PR(>F)']
                    sig = "***" if p_val < 0.001 else "**" if p_val < 0.01 else "*" if p_val < 0.05 else "n.s."
                    print(f"  {idx}: p={p_val:.4f} {sig}")
        except Exception as e:
            print(f"Could not perform two-way ANOVA: {e}")
else:
    print("Insufficient data or conditions for ANOVA.")

## 9. Reaction Time Analysis

### Exploratory Analysis 2 (RT): 
Examines reaction times across tasks and investigates speed-accuracy tradeoffs

In [ ]:
# ===== EXPLORATORY ANALYSIS 2: REACTION TIME ANALYSIS =====

print("\n" + "="*60)
print("REACTION TIME ANALYSIS")
print("="*60)

# --- VOWEL RT ANALYSIS ---
if not df_vowel.empty and 'rt' in df_vowel.columns:
    print("\nVowel Reaction Times:")
    print("-" * 60)
    
    # Remove outliers (RT > 10s likely invalid)
    df_v_clean = df_vowel[df_vowel['rt'] < 10].copy()
    
    # Speed-accuracy tradeoff
    correct_rt = df_v_clean[df_v_clean['score'] == 1]['rt'].values
    incorrect_rt = df_v_clean[df_v_clean['score'] == 0]['rt'].values
    
    print(f"Correct responses:   RT = {np.mean(correct_rt):.2f}s ± {np.std(correct_rt):.2f}s (n={len(correct_rt)})")
    print(f"Incorrect responses: RT = {np.mean(incorrect_rt):.2f}s ± {np.std(incorrect_rt):.2f}s (n={len(incorrect_rt)})")
    
    test_name, stat, p, cohens_d = stat_test_auto(correct_rt, incorrect_rt)
    print(f"Statistical Test: {test_name}, p={p:.4f}, d={cohens_d:.2f}")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # RT distribution by accuracy
    axes[0].violinplot([correct_rt, incorrect_rt], positions=[0, 1], widths=0.7,
                       showmeans=True, showextrema=True)
    axes[0].set_xticks([0, 1])
    axes[0].set_xticklabels(['Correct', 'Incorrect'])
    axes[0].set_ylabel('Reaction Time (s)', fontweight='bold')
    axes[0].set_title('Vowel RT by Response Accuracy\nSpeed-Accuracy Tradeoff Analysis')
    axes[0].grid(axis='y', alpha=0.3)
    
    # RT by phoneme difficulty
    rt_by_phoneme = df_v_clean.groupby('target_label').agg({
        'rt': ['mean', 'std', 'count'],
        'score': 'mean'
    }).reset_index()
    rt_by_phoneme.columns = ['Phoneme', 'RT_mean', 'RT_std', 'n', 'Accuracy']
    rt_by_phoneme = rt_by_phoneme.dropna().sort_values('RT_mean')
    
    ax2_twin = axes[1].twinx()
    
    x_pos = range(len(rt_by_phoneme))
    axes[1].bar(x_pos, rt_by_phoneme['RT_mean'], alpha=0.6, color='steelblue', label='Mean RT')
    ax2_twin.plot(x_pos, rt_by_phoneme['Accuracy'] * 100, 'ro-', linewidth=2, markersize=8, label='Accuracy')
    
    axes[1].set_xticks(x_pos)
    axes[1].set_xticklabels(rt_by_phoneme['Phoneme'], rotation=45)
    axes[1].set_ylabel('Reaction Time (s)', fontweight='bold', color='steelblue')
    ax2_twin.set_ylabel('Accuracy (%)', fontweight='bold', color='red')
    axes[1].set_xlabel('Vowel', fontweight='bold')
    axes[1].set_title('RT and Accuracy by Vowel')
    axes[1].legend(loc='upper left')
    ax2_twin.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

# --- CRM RT vs SNR CORRELATION (Analysis 7) ---
print("\n" + "="*60)
print("EXPLORATORY ANALYSIS 7: CRM RT vs SNR CORRELATION")
print("="*60)

if not df_crm.empty and 'rt' in df_crm.columns:
    # Clean data
    df_crm_clean = df_crm[(df_crm['rt'] < 10) & (df_crm['rt'] > 0) & (df_crm['snr'] < 20)].copy()
    
    # Correlation analysis
    r_pearson, p_pearson = pearsonr(df_crm_clean['snr'], df_crm_clean['rt'])
    r_spearman, p_spearman = spearmanr(df_crm_clean['snr'], df_crm_clean['rt'])
    
    print(f"\nCorrelation between SNR and Reaction Time:")
    print(f"  Pearson r = {r_pearson:.3f}, p = {p_pearson:.4f}")
    print(f"  Spearman rho = {r_spearman:.3f}, p = {p_spearman:.4f}")
    print(f"  n = {len(df_crm_clean)}")
    
    if p_spearman < 0.05:
        interpretation = "Lower SNR (harder trials) associated with " + ("LONGER" if r_spearman > 0 else "SHORTER") + " RTs"
    else:
        interpretation = "No significant relationship between SNR and RT"
    print(f"\nInterpretation: {interpretation}")
    
    # Plot
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter with regression
    axes[0].scatter(df_crm_clean['snr'], df_crm_clean['rt'], alpha=0.3, s=20)
    z = np.polyfit(df_crm_clean['snr'], df_crm_clean['rt'], 1)
    p_fit = np.poly1d(z)
    snr_range = np.linspace(df_crm_clean['snr'].min(), df_crm_clean['snr'].max(), 100)
    axes[0].plot(snr_range, p_fit(snr_range), 'r-', linewidth=2, 
                label=f'r={r_pearson:.2f}, p={p_pearson:.4f}')
    axes[0].set_xlabel('SNR (dB)', fontweight='bold')
    axes[0].set_ylabel('Reaction Time (s)', fontweight='bold')
    axes[0].set_title(f'CRM: RT vs SNR\n{interpretation}')
    axes[0].legend()
    axes[0].grid(alpha=0.3)
    
    # RT by correctness
    correct_crm_rt = df_crm_clean[df_crm_clean['correct']]['rt'].values
    incorrect_crm_rt = df_crm_clean[~df_crm_clean['correct']]['rt'].values
    
    parts = axes[1].violinplot([correct_crm_rt, incorrect_crm_rt], positions=[0, 1],
                               widths=0.7, showmeans=True, showextrema=True)
    axes[1].set_xticks([0, 1])
    axes[1].set_xticklabels(['Correct', 'Incorrect'])
    axes[1].set_ylabel('Reaction Time (s)', fontweight='bold')
    axes[1].set_title('CRM RT by Response Accuracy')
    axes[1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()

## 10. CRM Error Type Analysis

### Exploratory Analysis 8:
Breaks down CRM errors by type (Color, Number, or Both) and examines patterns across conditions and SNR levels

In [ ]:
# ===== EXPLORATORY ANALYSIS 8: CRM ERROR TYPE ANALYSIS =====

print("\n" + "="*60)
print("CRM ERROR TYPE STRATIFICATION")
print("="*60)

if not df_crm.empty:
    df_crm_valid = df_crm[~df_crm['condition'].isin(['Practice', 'Unknown'])].copy()
    
    # Overall error distribution
    print("\nOverall Error Distribution:")
    print("-" * 60)
    error_counts = df_crm_valid['error_type'].value_counts()
    for err_type, count in error_counts.items():
        pct = (count / len(df_crm_valid)) * 100
        print(f"{err_type:15s}: {count:4d} trials ({pct:5.1f}%)")
    
    # Error distribution by condition
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Stacked bar by condition
    error_by_cond = pd.crosstab(df_crm_valid['condition'], df_crm_valid['error_type'], normalize='index') * 100
    error_by_cond.plot(kind='bar', stacked=True, ax=axes[0,0], colormap='viridis', edgecolor='black')
    axes[0,0].set_ylabel('Percentage of Trials', fontweight='bold')
    axes[0,0].set_xlabel('Condition', fontweight='bold')
    axes[0,0].set_title('Error Type Distribution by Condition')
    axes[0,0].legend(title='Error Type', bbox_to_anchor=(1.05, 1))
    axes[0,0].set_xticklabels(axes[0,0].get_xticklabels(), rotation=45)
    
    # 2. Error-only breakdown (excluding correct)
    df_errors_only = df_crm_valid[df_crm_valid['error_type'] != 'Correct'].copy()
    
    if not df_errors_only.empty:
        error_only_dist = pd.crosstab(df_errors_only['condition'], df_errors_only['error_type'], normalize='index') * 100
        error_only_dist.plot(kind='bar', ax=axes[0,1], colormap='Set2', edgecolor='black')
        axes[0,1].set_ylabel('Percentage of Error Trials', fontweight='bold')
        axes[0,1].set_xlabel('Condition', fontweight='bold')
        axes[0,1].set_title('Error Type Among INCORRECT Trials Only')
        axes[0,1].legend(title='Error Type', bbox_to_anchor=(1.05, 1))
        axes[0,1].set_xticklabels(axes[0,1].get_xticklabels(), rotation=45)
    
    # 3. Error rate vs SNR
    # Bin SNR into ranges
    df_crm_valid['snr_bin'] = pd.cut(df_crm_valid['snr'], bins=[-20, -10, -5, 0, 5, 10, 20], 
                                      labels=['<-10', '-10 to -5', '-5 to 0', '0 to 5', '5 to 10', '>10'])
    
    snr_error = pd.crosstab(df_crm_valid['snr_bin'], df_crm_valid['error_type'], normalize='index') * 100
    snr_error.plot(kind='bar', stacked=False, ax=axes[1,0], colormap='tab10', edgecolor='black')
    axes[1,0].set_ylabel('Percentage of Trials', fontweight='bold')
    axes[1,0].set_xlabel('SNR Range (dB)', fontweight='bold')
    axes[1,0].set_title('Error Types by SNR Level')
    axes[1,0].legend(title='Error Type', bbox_to_anchor=(1.05, 1))
    axes[1,0].set_xticklabels(axes[1,0].get_xticklabels(), rotation=45)
    
    # 4. Specific error comparison: Color vs Number
    color_errors = df_errors_only[df_errors_only['error_type'] == 'Color Error']
    number_errors = df_errors_only[df_errors_only['error_type'] == 'Number Error']
    both_errors = df_errors_only[df_errors_only['error_type'] == 'Both Error']
    
    error_counts_comp = [len(color_errors), len(number_errors), len(both_errors)]
    colors = ['#FF6B6B', '#4ECDC4', '#95A99C']
    
    bars = axes[1,1].bar(['Color\nOnly', 'Number\nOnly', 'Both'], error_counts_comp, 
                        color=colors, edgecolor='black', linewidth=2)
    
    # Add percentages on bars
    total_errors = sum(error_counts_comp)
    for i, (bar, count) in enumerate(zip(bars, error_counts_comp)):
        pct = (count / total_errors) * 100 if total_errors > 0 else 0
        axes[1,1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(error_counts_comp)*0.02,
                      f'{count}\n({pct:.1f}%)', ha='center', va='bottom', fontweight='bold')
    
    axes[1,1].set_ylabel('Number of Errors', fontweight='bold')
    axes[1,1].set_title(f'Specific Error Type Counts\n(Total Errors: {total_errors})')
    axes[1,1].grid(axis='y', alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Statistical comparison
    print("\nClinical Interpretation:")
    print("-" * 60)
    if len(color_errors) > len(number_errors) * 1.5:
        print("→ Color errors predominate: Subject may have difficulty with")
        print("  temporal stream segregation or color keyword identification")
    elif len(number_errors) > len(color_errors) * 1.5:
        print("→ Number errors predominate: Subject may have difficulty with")
        print("  number keyword identification or memory")
    else:
        print("→ Balanced error distribution: No specific keyword weakness detected")

## 11. Cross-Task Performance Correlation

### Exploratory Analysis 9:
Examines relationships between vowel/consonant accuracy and CRM performance

In [ ]:
# ===== EXPLORATORY ANALYSIS 9: CROSS-TASK CORRELATION =====

print("\n" + "="*60)
print("CROSS-TASK PERFORMANCE CORRELATION")
print("="*60)

# Calculate aggregate scores per subject (or condition)
correlation_data = {}

# Vowel accuracy by condition
if not df_vowel.empty:
    for cond in df_vowel['condition'].unique():
        acc = df_vowel[df_vowel['condition'] == cond]['score'].mean()
        correlation_data[f'vowel_acc_{cond}'] = acc
    
    overall_vowel_acc = df_vowel['score'].mean()
    correlation_data['vowel_acc_overall'] = overall_vowel_acc

# Consonant accuracy
if not df_consonant.empty:
    overall_cons_acc = df_consonant['score'].mean()
    correlation_data['consonant_acc_overall'] = overall_cons_acc

# CRM SRT by condition
if not df_crm_summary.empty:
    for cond in df_crm_summary['condition'].unique():
        if cond not in ['Practice', 'Unknown']:
            mean_srt = df_crm_summary[df_crm_summary['condition'] == cond]['srt'].mean()
            correlation_data[f'crm_srt_{cond}'] = mean_srt

print("\nAggregate Performance Metrics:")
print("-" * 60)
for key, val in correlation_data.items():
    print(f"{key:30s}: {val:.3f}")

# For single-subject data, we can examine trial-level correlations instead
print("\n" + "="*60)
print("TRIAL-LEVEL CORRELATION: Vowel Accuracy vs RT")
print("="*60)

if not df_vowel.empty and len(df_vowel) > 10:
    # Examine if there's a temporal correlation (learning/fatigue)
    # Create bins of trials
    n_bins = min(10, len(df_vowel) // 20)
    df_vowel['trial_bin'] = pd.qcut(df_vowel['trial_idx'], q=n_bins, labels=False, duplicates='drop')
    
    bin_stats = df_vowel.groupby('trial_bin').agg({
        'score': ['mean', 'std', 'count'],
        'rt': 'mean'
    }).reset_index()
    bin_stats.columns = ['bin', 'acc_mean', 'acc_std', 'n', 'rt_mean']
    
    # Correlation across bins
    if len(bin_stats) > 3:
        r_temporal, p_temporal = pearsonr(bin_stats['bin'], bin_stats['acc_mean'])
        print(f"\nTemporal correlation (trial bin vs accuracy):")
        print(f"  r = {r_temporal:.3f}, p = {p_temporal:.4f}")
        
        if p_temporal < 0.05:
            if r_temporal > 0:
                print("  → Significant LEARNING effect detected")
            else:
                print("  → Significant FATIGUE effect detected")
        else:
            print("  → No significant temporal trend")
    
    # Visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Accuracy over trial bins
    axes[0].plot(bin_stats['bin'], bin_stats['acc_mean'] * 100, 'o-', linewidth=2, markersize=8)
    axes[0].fill_between(bin_stats['bin'], 
                         (bin_stats['acc_mean'] - bin_stats['acc_std']) * 100,
                         (bin_stats['acc_mean'] + bin_stats['acc_std']) * 100,
                         alpha=0.3)
    axes[0].set_xlabel('Trial Bin (Temporal Sequence)', fontweight='bold')
    axes[0].set_ylabel('Accuracy (%)', fontweight='bold')
    axes[0].set_title('Vowel Accuracy Over Time\n(Learning/Fatigue Analysis)')
    axes[0].grid(alpha=0.3)
    
    # RT over trial bins
    axes[1].plot(bin_stats['bin'], bin_stats['rt_mean'], 'o-', color='coral', linewidth=2, markersize=8)
    axes[1].set_xlabel('Trial Bin (Temporal Sequence)', fontweight='bold')
    axes[1].set_ylabel('Mean Reaction Time (s)', fontweight='bold')
    axes[1].set_title('Reaction Time Over Time')
    axes[1].grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Note about single-subject limitations
print("\n" + "="*60)
print("NOTE: Cross-Task Correlation Analysis")
print("="*60)
print("For robust cross-task correlation analysis, data from multiple")
print("subjects would be needed. With single-subject data, we focus on:")
print("  1. Within-task temporal trends (learning/fatigue)")
print("  2. Condition-specific performance patterns")
print("  3. Feature-level analysis (which features are most accessible)")

## 12. Temporal Trends: Learning and Fatigue

### Exploratory Analysis 10:
Comprehensive temporal analysis examining performance changes over the session

In [ ]:
# ===== EXPLORATORY ANALYSIS 10: COMPREHENSIVE TEMPORAL ANALYSIS =====

print("\n" + "="*60)
print("TEMPORAL TRENDS: LEARNING AND FATIGUE ANALYSIS")
print("="*60)

# --- CRM TEMPORAL ANALYSIS ---
if not df_crm.empty:
    df_crm_temporal = df_crm[~df_crm['condition'].isin(['Practice', 'Unknown'])].copy()
    
    print("\n1. CRM Performance Over Time")
    print("-" * 60)
    
    # Add global trial index (across all runs)
    df_crm_temporal = df_crm_temporal.sort_values(['run_id', 'trial_in_run']).reset_index(drop=True)
    df_crm_temporal['global_trial'] = range(len(df_crm_temporal))
    
    # Bin trials for clearer trends
    n_bins = min(20, len(df_crm_temporal) // 10)
    df_crm_temporal['trial_bin'] = pd.cut(df_crm_temporal['global_trial'], bins=n_bins, labels=False)
    
    temporal_stats = df_crm_temporal.groupby('trial_bin').agg({
        'snr': ['mean', 'std', 'count'],
        'correct': 'mean',
        'rt': 'mean'
    }).reset_index()
    temporal_stats.columns = ['bin', 'snr_mean', 'snr_std', 'n', 'acc', 'rt_mean']
    
    # Statistical trend test
    slope_snr, intercept, r_val, p_val, std_err = stats.linregress(temporal_stats['bin'], temporal_stats['snr_mean'])
    
    print(f"SNR trend over time:")
    print(f"  Slope: {slope_snr:.4f} dB per bin")
    print(f"  R² = {r_val**2:.3f}, p = {p_val:.4f}")
    
    if p_val < 0.05:
        if slope_snr < -0.1:
            print("  → Significant IMPROVEMENT detected (SNR decreasing over time)")
            print("     Subject needed less signal as session progressed (learning)")
        elif slope_snr > 0.1:
            print("  → Significant DECLINE detected (SNR increasing over time)")
            print("     Subject needed more signal as session progressed (fatigue)")
    else:
        print("  → No significant temporal trend (stable performance)")
    
    # Visualization
    fig = plt.figure(figsize=(16, 10))
    gs = fig.add_gridspec(3, 2, hspace=0.3, wspace=0.3)
    
    # 1. SNR over time with trend
    ax1 = fig.add_subplot(gs[0, :])
    ax1.scatter(df_crm_temporal['global_trial'], df_crm_temporal['snr'], 
               c=df_crm_temporal['correct'], cmap='RdYlGn', alpha=0.5, s=20)
    ax1.plot(temporal_stats['bin'] * len(df_crm_temporal) / n_bins, 
            temporal_stats['snr_mean'], 'b-', linewidth=3, label='Mean SNR (binned)')
    
    # Add trend line
    x_trend = np.array([0, len(df_crm_temporal)])
    y_trend = slope_snr * (x_trend / len(df_crm_temporal) * n_bins) + intercept
    ax1.plot(x_trend, y_trend, 'r--', linewidth=2, 
            label=f'Trend: slope={slope_snr:.3f} dB/bin, p={p_val:.4f}')
    
    ax1.set_xlabel('Global Trial Number', fontweight='bold')
    ax1.set_ylabel('SNR (dB)', fontweight='bold')
    ax1.set_title('CRM SNR Over Entire Session\n(Green=Correct, Red=Incorrect)', fontsize=14)
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # 2. Accuracy by run order
    ax2 = fig.add_subplot(gs[1, 0])
    run_order_stats = df_crm_temporal.groupby('run_id').agg({
        'correct': 'mean',
        'condition': 'first'
    }).reset_index()
    run_order_stats = run_order_stats.sort_values('run_id')
    
    colors_run = [plt.cm.Set2(i) for i in range(len(run_order_stats))]
    ax2.bar(range(len(run_order_stats)), run_order_stats['correct'] * 100, 
           color=colors_run, edgecolor='black', linewidth=1.5)
    ax2.set_xticks(range(len(run_order_stats)))
    ax2.set_xticklabels([f"R{int(r)}\n{c}" for r, c in zip(run_order_stats['run_id'], run_order_stats['condition'])],
                       rotation=45, ha='right')
    ax2.set_ylabel('Accuracy (%)', fontweight='bold')
    ax2.set_xlabel('Run ID & Condition', fontweight='bold')
    ax2.set_title('Accuracy by Run Order')
    ax2.set_ylim(0, 100)
    ax2.grid(axis='y', alpha=0.3)
    
    # 3. RT over time
    ax3 = fig.add_subplot(gs[1, 1])
    ax3.plot(temporal_stats['bin'], temporal_stats['rt_mean'], 'o-', 
            color='purple', linewidth=2, markersize=8)
    ax3.set_xlabel('Trial Bin', fontweight='bold')
    ax3.set_ylabel('Mean RT (s)', fontweight='bold')
    ax3.set_title('Reaction Time Across Session')
    ax3.grid(alpha=0.3)
    
    # 4. Accuracy over time
    ax4 = fig.add_subplot(gs[2, 0])
    ax4.plot(temporal_stats['bin'], temporal_stats['acc'] * 100, 'o-',
            color='green', linewidth=2, markersize=8)
    ax4.set_xlabel('Trial Bin', fontweight='bold')
    ax4.set_ylabel('Accuracy (%)', fontweight='bold')
    ax4.set_title('Accuracy Across Session')
    ax4.set_ylim(0, 100)
    ax4.grid(alpha=0.3)
    
    # 5. SNR by condition over time
    ax5 = fig.add_subplot(gs[2, 1])
    for cond in df_crm_temporal['condition'].unique():
        df_cond = df_crm_temporal[df_crm_temporal['condition'] == cond]
        if len(df_cond) > 5:
            # Create bins for this condition
            df_cond = df_cond.sort_values('global_trial').copy()
            df_cond['cond_trial'] = range(len(df_cond))
            window = max(3, len(df_cond) // 5)
            df_cond['snr_rolling'] = df_cond['snr'].rolling(window=window, center=True).mean()
            
            ax5.plot(df_cond['cond_trial'], df_cond['snr_rolling'], 
                    '-', linewidth=2, label=cond, alpha=0.8)
    
    ax5.set_xlabel('Trial Within Condition', fontweight='bold')
    ax5.set_ylabel('SNR (dB, rolling average)', fontweight='bold')
    ax5.set_title('SNR Trends by Condition')
    ax5.legend()
    ax5.grid(alpha=0.3)
    
    plt.suptitle('Comprehensive Temporal Analysis: Learning and Fatigue Detection', 
                fontsize=16, fontweight='bold', y=0.995)
    plt.show()

print("\n" + "="*60)
print("SUMMARY: Key Findings")
print("="*60)
print("\nThis completes the comprehensive 10-analysis exploratory pipeline.")
print("All analyses include:")
print("  ✓ Sample sizes (n) for all metrics")
print("  ✓ 95% Confidence intervals via bootstrap")
print("  ✓ Appropriate statistical tests with p-values and effect sizes")
print("  ✓ Publication-ready visualizations")
print("\nAnalyses completed:")
print("  1. Phonetic Feature Analysis (Place/Manner/Voicing)")
print("  2. Confusion Matrices (Vowels & Consonants)")
print("  3. Talker-Specific Performance")
print("  4. Granular CRM SRT Analysis")
print("  5. Statistical Significance Testing (ANOVA/Post-hoc)")
print("  6. Voice Gender Release from Masking (VGRM)")
print("  7. RT vs SNR Correlation")
print("  8. CRM Error Type Stratification")
print("  9. Cross-Task Performance Patterns")
print(" 10. Temporal Trends (Learning/Fatigue)")

## Analysis Complete

---

### Next Steps:
1. **Export Results**: Save figures using `plt.savefig()` for publications
2. **Statistical Tables**: Export dataframes to CSV for reporting
3. **Interpretation**: Review all p-values and confidence intervals for clinical significance

### Citation:
If using this pipeline, please cite the Miller-Nicely (1955) framework for phonetic feature analysis.

---

**Pipeline Version**: 4.0 - Claude Code Edition  
**Features**: Production-ready, statistically rigorous, publication-quality